In [1]:
import os
import json
import pandas as pd
import traceback

In [4]:
from langchain_openai import ChatOpenAI

In [7]:
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)